In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from rtseg.utils.param_io import load_params
from pathlib import Path
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
from rtseg.forkplot import compute_forkplot_stats
import matplotlib.patches as patches
%matplotlib qt5

In [3]:
from rtseg.segmentation import LiveNet, get_live_model, live_segment

/home/pk/Documents/rtclient/.venv/lib/python3.11/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: '2.0.3' (you have '2.0.0'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
params_path = Path("/home/pk/Documents/rtseg/rtseg/resources/reference_params/reference_linux.yaml")
params = load_params(params_path, ref_type='expt')

In [5]:
phase_path = Path('/home/pk/Documents/rtclient/rtclient/resources/test_images/phase_dummy.tiff')
fluor_path = Path('/home/pk/Documents/rtclient/rtclient/resources/test_images/fluor_dummy.tiff')
phase_img = imread(phase_path).astype('float32')
fluor_img = imread(fluor_path)
datapoint = {
    'phase': phase_img,
    'fluor': fluor_img,
    'position': 1,
    'timepoint': 0,
}

In [6]:
live_net = get_live_model(params)

In [7]:
seg_result = live_segment(datapoint, live_net, params)

In [8]:
plt.figure()
plt.imshow(seg_result['seg_mask'])
plt.show()

In [9]:
plt.figure()
plt.imshow(phase_img)
plt.show()

In [10]:
fig, ax = plt.subplots()
ax.imshow(phase_img, cmap='gray')
for row in seg_result['barcode_locations']:
    rect = patches.Rectangle((row[0], row[1]), row[2] - row[0], row[3] - row[1], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
plt.show()

### Dot coorindates

In [11]:
from rtseg.dotdetect import compute_dot_coordinates

In [12]:
params.Hardware.device

'cuda:0'

In [13]:
dots= compute_dot_coordinates(seg_result['fluor'], seg_result['seg_mask'], params)

In [14]:
dots

{'raw_coords': array([[ 790.09599631, 1478.81642799],
        [1264.70615296, 3027.29873491],
        [ 278.22508568, 2941.70431803],
        ...,
        [1014.99314642,  100.4847352 ],
        [ 168.46496874,   18.97020964],
        [ 170.55851776,   10.02274833]]),
 'rotated_coords': array([[1556.18357201,  790.09599631],
        [   7.70126509, 1264.70615296],
        [  93.29568197,  278.22508568],
        ...,
        [2934.5152648 , 1014.99314642],
        [3016.02979036,  168.46496874],
        [3024.97725167,  170.55851776]])}

In [15]:
def plot_dots(fluor_image, dots):
    plt.figure()
    plt.imshow(fluor_image)
    plt.plot(dots[:, 1], dots[:, 0], 'ro')
    plt.show()

In [16]:
plot_dots(seg_result['seg_mask'], dots['raw_coords'])

### Computing Internal coordinates to plot

In [17]:
from rtseg.cells.utils import regionprops_custom

In [18]:
trap_locations = seg_result['trap_locations_list']

In [19]:
fork_stats = compute_forkplot_stats(seg_result['seg_mask'], dots['rotated_coords'], trap_locations=trap_locations)

In [20]:
def transform_to_polars(stats):
    keys = stats.keys()

In [21]:
stats = fork_stats

In [22]:
stats[0].keys()

dict_keys(['position', 'timepoint', 'trap', 'cell_label', 'area', 'length', 'normalization_counts', 'internal_coord', 'normalized_internal_x', 'bbox', 'global_coords', 'local_coords'])

In [23]:
d = {key: [] for key in stats[0].keys()}

In [24]:
for i in range(len(stats)):
    for key, value in stats[i].items():
        d[key].append(value)

In [25]:
import polars as pl

In [26]:
df = pl.DataFrame(d)

In [27]:
df.schema

Schema([('position', Int64),
        ('timepoint', Int64),
        ('trap', Int64),
        ('cell_label', Int32),
        ('area', Float64),
        ('length', Float64),
        ('normalization_counts', Int64),
        ('internal_coord', List(Float64)),
        ('normalized_internal_x', Float64),
        ('bbox', List(Int64)),
        ('global_coords', List(Float64)),
        ('local_coords', List(Float64))])

In [28]:
df.shape

(901, 12)

In [29]:
import glob

In [30]:
glob.glob("/mnt/sda1/REALTIME/test_runs/Pos[0-9]*/forks.parquet")

['/mnt/sda1/REALTIME/test_runs/Pos15/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos5/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos29/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos2/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos3/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos10/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos21/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos17/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos26/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos8/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos11/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos24/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos23/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos20/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos22/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos6/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos13/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos12/forks.parquet',
 '/mnt/sda1/REALTIME/test_runs/Pos16/forks.parquet'

In [31]:
path = Path("/mnt/sda1/REALTIME/test_runs/Pos[0-999]/forks.parquet")


In [67]:
# Define the columns you want to extract
columns_to_extract = ['area', 'length', 'normalized_internal_x', 'normalization_counts']

# Read the Parquet files and extract specified columns
#df = pl.read_parquet(glob.glob("/mnt/sda1/REALTIME/test_runs/Pos[0-9]*/forks.parquet"), columns=columns_to_extract)
df = pl.read_parquet(glob.glob("/mnt/sda1/REALTIME/test_runs/Pos[0-9]*/forks.parquet"))
# Display the resulting DataFrame
print(df)

shape: (567_630, 12)
┌──────────┬───────────┬──────┬────────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ position ┆ timepoint ┆ trap ┆ cell_label ┆ … ┆ normalized ┆ bbox       ┆ global_coo ┆ local_coor │
│ ---      ┆ ---       ┆ ---  ┆ ---        ┆   ┆ _internal_ ┆ ---        ┆ rds        ┆ ds         │
│ i64      ┆ i64       ┆ i64  ┆ i32        ┆   ┆ x          ┆ list[i64]  ┆ ---        ┆ ---        │
│          ┆           ┆      ┆            ┆   ┆ ---        ┆            ┆ list[f64]  ┆ list[f64]  │
│          ┆           ┆      ┆            ┆   ┆ f64        ┆            ┆            ┆            │
╞══════════╪═══════════╪══════╪════════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 15       ┆ 0         ┆ 0    ┆ 20         ┆ … ┆ 0.183908   ┆ [2898,     ┆ [2903.0426 ┆ [5.042699, │
│          ┆           ┆      ┆            ┆   ┆            ┆ 231, …     ┆ 99, 245.74 ┆ 14.743775] │
│          ┆           ┆      ┆            ┆   ┆            ┆ 309]    

In [68]:
df.schema

Schema([('position', Int64),
        ('timepoint', Int64),
        ('trap', Int64),
        ('cell_label', Int32),
        ('area', Float64),
        ('length', Float64),
        ('normalization_counts', Int64),
        ('internal_coord', List(Float64)),
        ('normalized_internal_x', Float64),
        ('bbox', List(Int64)),
        ('global_coords', List(Float64)),
        ('local_coords', List(Float64))])

### Read pandas and convert to parquet

In [3]:
import pandas as pd
from pathlib import Path
import polars as pl
import ast
import glob

# Define a custom converter function
def safe_literal_eval(value):
    try:
        return ast.literal_eval(value)
    except (SyntaxError, ValueError):
        return None  # or any other default value

In [4]:
save_dir = Path("F:Oscar/EXP-25-CD0117/live_run/")

In [10]:
fork_filenames = glob.glob(str(save_dir / Path('Pos[0-9]*/forks.parquet')))
columns_to_extract = ['area', 'length', 'normalized_internal_x', 'normalization_counts']
data = pl.read_parquet(fork_filenames, use_pyarrow=True, columns=columns_to_extract)


In [11]:
data.shape

(9591797, 4)

In [7]:

forks_filenames = [position_dir / Path('forks.csv') for position_dir in list(save_dir.glob('Pos*'))]
schema = {
        'position':pl.Int64,
        'timepoint':pl.Int64,
        'trap' : pl.Int64,
        'cell_label': pl.Int32,
        'area' : pl.Float64,
        'length' : pl.Float64,
        'normalization_counts' :pl.Int64,
        'internal_coord': pl.List(pl.Float64),
        'normalized_internal_x': pl.Float64,
        'bbox' : pl.List(pl.Int64),
        'global_coords' : pl.List(pl.Float64),
        'local_coords' : pl.List(pl.Float64)
}
for filename in forks_filenames:
    dataframes = pd.read_csv(filename, converters= {'internal_coord': safe_literal_eval,
                                                             'bbox': safe_literal_eval,
                                                             'global_coords': safe_literal_eval,
                                                             'local_coords': safe_literal_eval})
    df2 = pl.from_pandas(dataframes, schema_overrides=schema)
    write_path = filename.parent / Path('forks.parquet')
    df2.write_parquet(write_path, use_pyarrow=True, partition_by='trap')
    print(write_path, df2.shape)

F:Oscar\EXP-25-CD0117\live_run\Pos1\forks.parquet (347576, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos10\forks.parquet (314135, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos11\forks.parquet (301068, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos12\forks.parquet (333604, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos13\forks.parquet (332025, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos14\forks.parquet (299641, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos15\forks.parquet (321389, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos16\forks.parquet (326609, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos17\forks.parquet (318428, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos18\forks.parquet (314377, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos19\forks.parquet (342689, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos2\forks.parquet (289667, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos20\forks.parquet (336885, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos21\forks.parquet (308606, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos22\forks.parquet (322492, 12)
F:Oscar\EXP-25-CD0117\live_run\Pos23\forks

In [ ]:
s

In [12]:
write_path

WindowsPath('F:Oscar/EXP-25-CD0117/live_run/Pos1/forks.parquet')

In [6]:
filename

WindowsPath('F:Oscar/EXP-25-CD0117/live_run/Pos1/forks.csv')

In [5]:
filename.parent

WindowsPath('F:Oscar/EXP-25-CD0117/live_run/Pos1')

In [6]:
df2.write_parquet(write_path, use_pyarrow=True, partition_by="trap")

PanicException: called `Result::unwrap()` on an `Err` value: ()

In [64]:
schema = {
        'position':pl.Int64,
        'timepoint':pl.Int64,
        'trap' : pl.Int64,
        'cell_label': pl.Int32,
        'area' : pl.Float64,
        'length' : pl.Float64,
        'normalization_counts' :pl.Int64,
        'internal_coord': pl.List(pl.Float64),
        'normalized_internal_x': pl.Float64,
        'bbox' : pl.List(pl.Int64),
        'global_coords' : pl.List(pl.Float64),
        'local_coords' : pl.List(pl.Float64)
}

In [69]:
df2 = pl.from_pandas(data, schema_overrides=schema)

In [72]:
df2.schema

Schema([('position', Int64),
        ('timepoint', Int64),
        ('trap', Int64),
        ('cell_label', Int32),
        ('area', Float64),
        ('length', Float64),
        ('normalization_counts', Int64),
        ('internal_coord', List(Float64)),
        ('normalized_internal_x', Float64),
        ('bbox', List(Int64)),
        ('global_coords', List(Float64)),
        ('local_coords', List(Float64))])

In [73]:
df.schema

Schema([('position', Int64),
        ('timepoint', Int64),
        ('trap', Int64),
        ('cell_label', Int32),
        ('area', Float64),
        ('length', Float64),
        ('normalization_counts', Int64),
        ('internal_coord', List(Float64)),
        ('normalized_internal_x', Float64),
        ('bbox', List(Int64)),
        ('global_coords', List(Float64)),
        ('local_coords', List(Float64))])

#### Testing reading parquets

In [67]:
import polars as pl

In [74]:
df = pl.read_parquet('/mnt/sda1/REALTIME/test_runs/Pos15/forks.parquet/')

In [75]:
df

position,timepoint,trap,cell_label,area,length,normalization_counts,internal_coord,normalized_internal_x,bbox,global_coords,local_coords
i64,i64,i64,i32,f64,f64,i64,list[f64],f64,list[i64],list[f64],list[f64]
15,0,0,20,969.0,77.473008,4,"[14.247897, 2.203242]",0.183908,"[2898, 231, … 309]","[2903.042699, 245.743775]","[5.042699, 14.743775]"
15,0,0,20,969.0,77.473008,4,"[5.598788, 3.350511]",0.072268,"[2898, 231, … 309]","[2902.018189, 236.998875]","[4.018189, 5.998875]"
15,0,0,20,969.0,77.473008,4,"[57.219483, -1.177507]",0.738573,"[2898, 231, … 309]","[2909.522023, 288.395867]","[11.522023, 57.395867]"
15,0,0,20,969.0,77.473008,4,"[52.055878, -2.792716]",0.671923,"[2898, 231, … 309]","[2910.430389, 283.094935]","[12.430389, 52.094935]"
15,0,0,55,808.0,72.518836,2,"[16.03578, -3.551992]",0.221126,"[2901, 312, … 385]","[2910.29358, 328.283664]","[9.29358, 16.283664]"
…,…,…,…,…,…,…,…,…,…,…,…
15,13,9,285,502.0,45.683252,2,"[34.396858, -3.359979]",0.752942,"[1999, 936, … 984]","[2012.054529, 970.844598]","[13.054529, 34.844598]"
15,13,9,298,1063.0,91.301096,2,"[29.361403, -1.199828]",0.321589,"[2000, 983, … 1077]","[2009.051945, 1012.519718]","[9.051945, 29.519718]"
15,13,9,298,1063.0,91.301096,2,"[47.641851, -6.248417]",0.52181,"[2000, 983, … 1077]","[2015.490588, 1030.496327]","[15.490588, 47.496327]"


In [35]:
import pandas as pd

In [36]:
fork_stats_table = pd.DataFrame(fork_stats)

In [37]:
fork_stats_table

,position,timepoint,trap,cell_label,area,length,normalization_counts,internal_coord,normalized_internal_x,bbox,global_coords,local_coords
0,0,0,4,15,368.0,30.025607,1,"(10.331315843264065, -2.273029129056189)",0.344083,"(2490, 227, 2511, 258)","(2502.3362740292982, 237.55769754397429)","(12.336274029298238, 10.557697543974285)"
1,0,0,8,16,596.0,55.741157,4,"(48.513692552829426, 4.7893035480254476)",0.870339,"(2082, 228, 2104, 286)","(2083.5672611944983, 277.55680462224456)","(1.5672611944983146, 49.556804622244556)"
2,0,0,8,16,596.0,55.741157,4,"(38.77562401109707, 3.410713575993412)",0.695637,"(2082, 228, 2104, 286)","(2085.3359853121174, 267.1814565483476)","(3.335985312117373, 39.181456548347626)"
3,0,0,8,16,596.0,55.741157,4,"(25.49408365838375, 1.4085934599614878)",0.457366,"(2082, 228, 2104, 286)","(2090.4882352941177, 254.24359861591697)","(8.488235294117658, 26.243598615916966)"
4,0,0,8,16,596.0,55.741157,4,"(17.678084451145736, -2.1620871565639836)",0.317146,"(2082, 228, 2104, 286)","(2096.785154092566, 248.4542140726831)","(14.78515409256579, 20.4542140726831)"
...,...,...,...,...,...,...,...,...,...,...,...,...
896,0,0,23,355,582.0,47.668006,1,"(24.969438280041942, 0.11646185491855153)",0.523820,"(535, 1198, 552, 1247)","(541.9235198311655, 1222.8753698988369)","(6.9235198311655495, 24.87536989883688)"
897,0,0,19,356,371.0,37.364992,2,"(18.515859724714527, 0.49341423287447705)",0.495540,"(944, 1204, 963, 1241)","(952.9429480044522, 1221.7819049133407)","(8.942948004452205, 17.781904913340668)"
898,0,0,19,356,371.0,37.364992,2,"(28.673333550869668, 2.5895484369136805)",0.767385,"(944, 1204, 963, 1241)","(954.3035876889455, 1232.0016390457113)","(10.303587688945527, 28.001639045711272)"
899,0,0,25,357,455.0,44.946226,2,"(38.345571980881836, 3.637052744213372)",0.853143,"(331, 1215, 350, 1259)","(340.2880712321592, 1252.842084588189)","(9.288071232159211, 37.84208458818898)"


In [38]:
len(fork_stats_table)

901

In [39]:
fork_stats_table.to_csv('forks.csv', mode='a', index=False, header=False)

In [40]:
df = pd.read_csv('forks.csv')

In [41]:
len(df)

3077

In [58]:
df.iloc[1025]

position                              position
timepoint                            timepoint
trap                                      trap
cell_label                          cell_label
area                                      area
length                                  length
normalization_counts      normalization_counts
internal_coord                  internal_coord
normalized_internal_x    normalized_internal_x
bbox                                      bbox
global_coords                    global_coords
local_coords                      local_coords
Name: 1025, dtype: object

In [54]:
trap_locations_left = [loc-40 for loc in trap_locations]
trap_locations_right = [loc+40 for loc in trap_locations]

In [58]:
print(trap_locations_left)
print(trap_locations_right)

[76, 178, 280, 382, 484, 588, 688, 790, 893, 995, 1097, 1199, 1300, 1404, 1543, 1646, 1747, 1851, 1956, 2055, 2156, 2259, 2364, 2466, 2566, 2668, 2771, 2873]
[156, 258, 360, 462, 564, 668, 768, 870, 973, 1075, 1177, 1279, 1380, 1484, 1623, 1726, 1827, 1931, 2036, 2135, 2236, 2339, 2444, 2546, 2646, 2748, 2851, 2953]


In [51]:
props[0].centroid

(2933.2462686567164, 167.36567164179104)

In [52]:
import bisect

In [61]:
for prop in props:
    left_index = bisect.bisect_left(trap_locations_left, prop.centroid[0])
    right_index = bisect.bisect_right(trap_locations_right, prop.centroid[0])
    print(prop.centroid[0], trap_locations_left[left_index-1], trap_locations_right[right_index], left_index-1, right_index)
    if left_index - 1 == right_index:
        prop.trap = right_index
    else:
        prop.trap = None
    prop.left_index = left_index
    prop.right_index = right_index

2933.2462686567164 2873 2953 27 27
2063.690476190476 2055 2135 19 19
2722.548672566372 2668 2748 25 25
1464.5013698630137 1404 1484 13 13
2207.8502673796793 2156 2236 20 20
1414.361963190184 1404 1484 13 13
2705.904593639576 2668 2748 25 25
1902.078313253012 1851 1931 17 17
1991.3779193205944 1956 2036 18 18
1600.5 1543 1623 14 14
2398.1688311688313 2364 2444 22 22
1347.2350119904077 1300 1380 12 12
2599.437984496124 2566 2646 24 24
800.2452830188679 790 870 7 7
2498.6603260869565 2466 2546 23 23
2091.6593959731545 2055 2135 19 19
1235.9347826086957 1199 1279 11 11
2804.16765578635 2771 2851 26 26
1149.392857142857 1097 1177 10 10
2906.8090815273476 2873 2953 27 27
1036.440281030445 995 1075 9 9
2190.878306878307 2156 2236 20 20
430.7031630170316 382 462 3 3
1680.3676470588234 1646 1726 15 15
1589.2236842105262 1543 1623 14 14
523.6628477905074 484 564 4 4
2295.34179357022 2259 2339 21 21
626.2301458670988 588 668 5 5
2599.326530612245 2566 2646 24 24
1887.576 1851 1931 17 17
1782.9785

In [56]:
props[0].left_index

28

In [57]:
props[0].right_index

27